# Cardiac Admission Prediction - Demo

## Demo dự đoán nguy cơ nhập viện tim mạch

**Mục tiêu:** Sử dụng model đã train để dự đoán nguy cơ nhập viện cho bệnh nhân mới

**Input:** Thông tin y tế của bệnh nhân (11 features)

**Output:**
- 0: Không có nguy cơ nhập viện
- 1: Có nguy cơ nhập viện tim mạch

## 1. Setup & Load Model

In [7]:
import os
import shutil

try:
    from google.colab import drive
    drive.mount('/content/drive')

    DRIVE_MODEL = '/content/drive/MyDrive/temp/bigdata/demo/cardiac_rf_model'

    if os.path.exists(DRIVE_MODEL):
        if os.path.exists('/content/cardiac_rf_model'):
            shutil.rmtree('/content/cardiac_rf_model')
        shutil.copytree(DRIVE_MODEL, '/content/cardiac_rf_model')
        print(f"✅ Model copied: {DRIVE_MODEL}")
    else:
        print(f"❌ Model not found: {DRIVE_MODEL}")
        print("Update DRIVE_MODEL variable with correct path")

except ImportError:
    print(f"Local mode: {os.getcwd()}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Model copied: /content/drive/MyDrive/temp/bigdata/demo/cardiac_rf_model


In [8]:
from pyspark.sql import SparkSession
from pyspark.ml import PipelineModel
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType

spark = SparkSession.builder \
    .appName("Cardiac Prediction Demo") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()

print(f"✅ Spark {spark.version} ready")

✅ Spark 3.5.1 ready


In [9]:
import os

MODEL_PATHS = [
    "cardiac_rf_model",
    "/content/cardiac_rf_model",
    "./cardiac_rf_model"
]

model_path = next((p for p in MODEL_PATHS if os.path.exists(p)), None)

if not model_path:
    raise FileNotFoundError("Model not found. Run 1_model_training.ipynb first or upload from Drive")

contents = os.listdir(model_path)
print(f"Found: {model_path}")
print(f"Contents: {contents}")

if 'metadata' not in contents or 'stages' not in contents:
    print("\n❌ WRONG MODEL TYPE!")
    print("This folder contains Keras/LSTM model files:")
    for f in contents:
        print(f"  - {f}")

    print("\n✅ Expected PySpark RandomForest model structure:")
    print("  cardiac_rf_model/")
    print("    ├── metadata/")
    print("    ├── stages/")
    print("    │   ├── 0_VectorAssembler_.../")
    print("    │   ├── 1_StandardScaler_.../")
    print("    │   ├── 2_VectorAssembler_.../")
    print("    │   └── 3_RandomForestClassifier_.../")

    print("\n🔧 TO FIX:")
    print("1. Run 1_model_training.ipynb completely")
    print("2. Delete this folder: rm -rf cardiac_rf_model")
    print("3. The notebook will create correct PySpark model")
    print("4. Copy to Drive: cp -r cardiac_rf_model /content/drive/MyDrive/")

    raise ValueError("Wrong model format - need PySpark model, found Keras model")

model = PipelineModel.load(model_path)
print(f"✅ Model loaded: {len(model.stages)} stages")

Found: cardiac_rf_model
Contents: ['stages', 'metadata']
✅ Model loaded: 4 stages


## 2. Hàm Predict

In [10]:
SCHEMA = StructType([
    StructField("heart_rate", DoubleType(), True),
    StructField("bp_sys", DoubleType(), True),
    StructField("bp_dia", DoubleType(), True),
    StructField("spo2", DoubleType(), True),
    StructField("resp_rate", DoubleType(), True),
    StructField("age", IntegerType(), True),
    StructField("sex", IntegerType(), True),
    StructField("hypertension_history", IntegerType(), True),
    StructField("diabetes_history", IntegerType(), True),
    StructField("heart_failure_history", IntegerType(), True),
    StructField("smoking_status", IntegerType(), True)
])

def predict_admission_risk(patient_record):
    test_df = spark.createDataFrame([patient_record], schema=SCHEMA)
    result = model.transform(test_df).select('prediction', 'probability').collect()[0]

    prediction = int(result['prediction'])
    probability = float(result['probability'][1])

    if prediction == 1:
        risk = "CAO" if probability > 0.7 else "TRUNG BÌNH"
        msg = f"⚠️ NGUY CƠ {risk} ({probability*100:.1f}%)"
    else:
        msg = f"✅ NGUY CƠ THẤP ({probability*100:.1f}%)"

    return prediction, probability, msg

print("✅ predict_admission_risk() ready")

✅ predict_admission_risk() ready


## 3. Demo Case 1: Bệnh nhân Nguy cơ Cao

In [11]:
high_risk = {
    'heart_rate': 135.0, 'bp_sys': 175.0, 'bp_dia': 105.0, 'spo2': 87.0,
    'resp_rate': 28.0, 'age': 78, 'sex': 1, 'hypertension_history': 1,
    'diabetes_history': 1, 'heart_failure_history': 1, 'smoking_status': 1
}

print("CASE 1: HIGH RISK PATIENT")
print(f"Age: {high_risk['age']}, Sex: {'M' if high_risk['sex'] else 'F'}")
print(f"Vitals: HR={high_risk['heart_rate']}, BP={high_risk['bp_sys']}/{high_risk['bp_dia']}, SpO2={high_risk['spo2']}")
print(f"History: HTN={high_risk['hypertension_history']}, DM={high_risk['diabetes_history']}, HF={high_risk['heart_failure_history']}, Smoke={high_risk['smoking_status']}")

pred, prob, msg = predict_admission_risk(high_risk)
print(f"\nResult: {msg}")

CASE 1: HIGH RISK PATIENT
Age: 78, Sex: M
Vitals: HR=135.0, BP=175.0/105.0, SpO2=87.0
History: HTN=1, DM=1, HF=1, Smoke=1

Result: ⚠️ NGUY CƠ CAO (88.7%)


## 4. Demo Case 2: Bệnh nhân Nguy cơ Thấp

In [12]:
low_risk = {
    'heart_rate': 72.0, 'bp_sys': 118.0, 'bp_dia': 78.0, 'spo2': 98.0,
    'resp_rate': 16.0, 'age': 28, 'sex': 0, 'hypertension_history': 0,
    'diabetes_history': 0, 'heart_failure_history': 0, 'smoking_status': 0
}

print("CASE 2: LOW RISK PATIENT")
print(f"Age: {low_risk['age']}, Sex: {'M' if low_risk['sex'] else 'F'}")
print(f"Vitals: HR={low_risk['heart_rate']}, BP={low_risk['bp_sys']}/{low_risk['bp_dia']}, SpO2={low_risk['spo2']}")
print(f"History: HTN={low_risk['hypertension_history']}, DM={low_risk['diabetes_history']}, HF={low_risk['heart_failure_history']}, Smoke={low_risk['smoking_status']}")

pred, prob, msg = predict_admission_risk(low_risk)
print(f"\nResult: {msg}")

CASE 2: LOW RISK PATIENT
Age: 28, Sex: F
Vitals: HR=72.0, BP=118.0/78.0, SpO2=98.0
History: HTN=0, DM=0, HF=0, Smoke=0

Result: ✅ NGUY CƠ THẤP (0.0%)


## 5. Demo Case 3: Bệnh nhân Nguy cơ Trung bình

In [13]:
medium_risk = {
    'heart_rate': 95.0, 'bp_sys': 145.0, 'bp_dia': 88.0, 'spo2': 94.0,
    'resp_rate': 20.0, 'age': 55, 'sex': 1, 'hypertension_history': 1,
    'diabetes_history': 0, 'heart_failure_history': 0, 'smoking_status': 1
}

print("CASE 3: MEDIUM RISK PATIENT")
print(f"Age: {medium_risk['age']}, Sex: {'M' if medium_risk['sex'] else 'F'}")
print(f"Vitals: HR={medium_risk['heart_rate']}, BP={medium_risk['bp_sys']}/{medium_risk['bp_dia']}, SpO2={medium_risk['spo2']}")
print(f"History: HTN={medium_risk['hypertension_history']}, DM={medium_risk['diabetes_history']}, HF={medium_risk['heart_failure_history']}, Smoke={medium_risk['smoking_status']}")

pred, prob, msg = predict_admission_risk(medium_risk)
print(f"\nResult: {msg}")

CASE 3: MEDIUM RISK PATIENT
Age: 55, Sex: M
Vitals: HR=95.0, BP=145.0/88.0, SpO2=94.0
History: HTN=1, DM=0, HF=0, Smoke=1

Result: ✅ NGUY CƠ THẤP (0.7%)


## 6. Batch Prediction - Dự đoán Nhiều Bệnh nhân

In [14]:
batch_patients = [
    high_risk,
    low_risk,
    medium_risk,
    {'heart_rate': 110.0, 'bp_sys': 160.0, 'bp_dia': 95.0, 'spo2': 90.0,
     'resp_rate': 22.0, 'age': 68, 'sex': 0, 'hypertension_history': 1,
     'diabetes_history': 1, 'heart_failure_history': 0, 'smoking_status': 0},
    {'heart_rate': 68.0, 'bp_sys': 115.0, 'bp_dia': 75.0, 'spo2': 99.0,
     'resp_rate': 14.0, 'age': 35, 'sex': 1, 'hypertension_history': 0,
     'diabetes_history': 0, 'heart_failure_history': 0, 'smoking_status': 0}
]

batch_df = spark.createDataFrame(batch_patients, schema=SCHEMA)
predictions = model.transform(batch_df)

print("BATCH PREDICTIONS")
results = predictions.select('age', 'sex', 'heart_rate', 'bp_sys', 'bp_dia', 'prediction', 'probability').collect()

for i, row in enumerate(results, 1):
    prob = float(row['probability'][1]) * 100
    risk = "HIGH" if row['prediction'] == 1 else "LOW"
    print(f"Patient #{i}: Age={row['age']}, Sex={'M' if row['sex'] else 'F'}, HR={row['heart_rate']}, BP={row['bp_sys']}/{row['bp_dia']}")
    print(f"  → {risk} RISK ({prob:.1f}%)\n")

BATCH PREDICTIONS
Patient #1: Age=78, Sex=M, HR=135.0, BP=175.0/105.0
  → HIGH RISK (88.7%)

Patient #2: Age=28, Sex=F, HR=72.0, BP=118.0/78.0
  → LOW RISK (0.0%)

Patient #3: Age=55, Sex=M, HR=95.0, BP=145.0/88.0
  → LOW RISK (0.7%)

Patient #4: Age=68, Sex=F, HR=110.0, BP=160.0/95.0
  → HIGH RISK (66.0%)

Patient #5: Age=35, Sex=M, HR=68.0, BP=115.0/75.0
  → LOW RISK (0.0%)



## 7. Giải thích Yếu tố Nguy cơ

### Các yếu tố nguy cơ chính dẫn đến nhập viện tim mạch:

#### 🔴 **Chỉ số sức khỏe bất thường:**
- **Nhịp tim** quá cao (>100 bpm) hoặc quá thấp (<60 bpm)
- **Huyết áp** cao (BP sys >140 mmHg, BP dia >90 mmHg)
- **SpO2** thấp (<92%) - thiếu oxy
- **Nhịp thở** nhanh (>20 lần/phút)

#### 🔴 **Yếu tố nhân khẩu học:**
- **Tuổi cao** (>65 tuổi) - nguy cơ tăng đáng kể
- **Giới tính** (Nam có xu hướng nguy cơ cao hơn)

#### 🔴 **Tiền sử bệnh lý:**
- **Tăng huyết áp**: Tăng nguy cơ biến chứng tim mạch
- **Tiểu đường**: Ảnh hưởng đến mạch máu và tim
- **Suy tim**: Tiền sử suy tim là yếu tố nguy cơ cao
- **Hút thuốc**: Làm tổn thương mạch máu

#### ✅ **Khuyến nghị:**
- Theo dõi thường xuyên các chỉ số sức khỏe
- Kiểm soát tốt các bệnh nền
- Thay đổi lối sống (bỏ thuốc lá, ăn uống lành mạnh, tập thể dục)
- Khám định kỳ với bác sĩ tim mạch

## 8. Kết luận

✅ **Hệ thống dự đoán nguy cơ nhập viện tim mạch đã hoàn thành!**

**Tính năng chính:**
- Load model đã được train từ `1_model_training.ipynb`
- Hàm `predict_admission_risk()` để dự đoán cho từng bệnh nhân
- Demo với các case nguy cơ cao, thấp, trung bình
- Batch prediction cho nhiều bệnh nhân cùng lúc
- Giải thích xác suất và yếu tố nguy cơ

**Ứng dụng thực tế:**
- Hỗ trợ bác sĩ đánh giá nhanh nguy cơ bệnh nhân
- Ưu tiên xử lý bệnh nhân có nguy cơ cao
- Theo dõi xu hướng sức khỏe theo thời gian
- Cảnh báo sớm biến chứng tim mạch

**Lưu ý:**
- Model chỉ mang tính tham khảo, không thay thế chẩn đoán của bác sĩ
- Cần update model định kỳ với dữ liệu mới
- Xem xét kỹ các ca có xác suất gần ngưỡng (40-60%)